In [1]:
import gym
env = gym.make('Marvin-v0')

In [6]:
env.reset()
for _ in range(100):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    if done:
        print("Finished")
        break
    print(f"Reward: {reward}")
env.close()

Reward: -0.13762489892790833
Reward: -0.20637878893315792
Reward: -0.15790618913372242
Reward: -0.3789379372497424
Reward: -0.22475865572690965
Reward: -0.1754904067913667
Reward: -0.008812867005668468
Reward: -0.03392187080780428
Reward: -0.059097557783126836
Reward: 0.007622751712799065
Reward: -0.13394125441710272
Reward: -0.31743144806225976
Reward: -0.18690743261575699
Reward: -0.1893915399238467
Reward: 0.010843021909393478
Reward: -0.18146576722462854
Reward: -0.20371100234985354
Reward: -0.13430123682816705
Reward: -0.1717740759551525
Reward: -0.2468811103105545
Reward: -0.2877690143585205
Reward: -0.2447798206806183
Reward: -0.2647436904410533
Reward: -0.22539074349403382
Reward: -0.4860661972915587
Reward: -0.2979118184447289
Reward: -0.05417288200060645
Reward: -0.14792812657356263
Reward: -0.09387289760013302
Reward: -0.13516389347612856
Reward: -0.10304445588588713
Reward: -0.23489382645487786
Reward: -0.13558241494993487
Reward: -0.17930668520927429
Reward: -0.26155313814

In [11]:
env.action_space.low, env.action_space.high

(array([-1., -1., -1., -1.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32))

In [13]:
env.observation_space.low, env.observation_space.high

(array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf], dtype=float32),
 array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       dtype=float32))